In [844]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jainaru/thyroid-disease-data")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/jainaru/thyroid-disease-data/versions/1


In [845]:
import os
os.listdir(path)

['Thyroid_Diff.csv']

In [846]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [847]:
df = pd.read_csv(f'{path}/Thyroid_Diff.csv')
df.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Indeterminate,No
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,T1a,N0,M0,I,Excellent,No


In [848]:
df.shape

(383, 17)

In [849]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Age                   383 non-null    int64 
 1   Gender                383 non-null    object
 2   Smoking               383 non-null    object
 3   Hx Smoking            383 non-null    object
 4   Hx Radiothreapy       383 non-null    object
 5   Thyroid Function      383 non-null    object
 6   Physical Examination  383 non-null    object
 7   Adenopathy            383 non-null    object
 8   Pathology             383 non-null    object
 9   Focality              383 non-null    object
 10  Risk                  383 non-null    object
 11  T                     383 non-null    object
 12  N                     383 non-null    object
 13  M                     383 non-null    object
 14  Stage                 383 non-null    object
 15  Response              383 non-null    ob

In [850]:
df['T'].unique()

array(['T1a', 'T1b', 'T2', 'T3a', 'T3b', 'T4a', 'T4b'], dtype=object)

In [851]:
enc = OneHotEncoder(categories='auto')
t = df['T'].values.reshape(-1, 1)
enc.fit(t)
new_features = enc.get_feature_names_out()
print(new_features)
new_t = pd.DataFrame(enc.transform(t).toarray())

['x0_T1a' 'x0_T1b' 'x0_T2' 'x0_T3a' 'x0_T3b' 'x0_T4a' 'x0_T4b']


In [852]:
new_t.columns = ['T1a', 'T1b', 'T2', 'T3a', 'T3b', 'T4a', 'T4b']

In [853]:
df = pd.concat([df, new_t], axis=1)
df.drop('T', axis=1, inplace=True)

In [854]:
df.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,...,Stage,Response,Recurred,T1a,T1b,T2,T3a,T3b,T4a,T4b
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,...,I,Indeterminate,No,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,...,I,Excellent,No,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,...,I,Excellent,No,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,...,I,Excellent,No,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,...,I,Excellent,No,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [855]:
df['Recurred'].unique()

array(['No', 'Yes'], dtype=object)

In [856]:
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
df['Smoking'] = df['Smoking'].map({'No': 0, 'Yes': 1})
df['Hx Smoking'] = df['Hx Smoking'].map({'No': 0, 'Yes': 1})
df['Hx Radiothreapy'] = df['Hx Radiothreapy'].map({'No': 0, 'Yes': 1})
df['Recurred'] = df['Recurred'].map({'No': 0, 'Yes': 1})


In [857]:
enc = OneHotEncoder(categories='auto')
thyroid = df['Thyroid Function'].values.reshape(-1, 1)
enc.fit(thyroid)
new_features = enc.get_feature_names_out()
print(new_features)
new_thyroid = pd.DataFrame(enc.transform(thyroid).toarray())

['x0_Clinical Hyperthyroidism' 'x0_Clinical Hypothyroidism' 'x0_Euthyroid'
 'x0_Subclinical Hyperthyroidism' 'x0_Subclinical Hypothyroidism']


In [858]:
new_thyroid.columns = ['Clinical Hyperthyroidism', 'Clinical Hypothyroidism', 'Euthyroid', 'Subclinical Hyperthyroidism', 'Subclinical Hypothyroidism']

In [859]:
df = pd.concat([df, new_thyroid], axis=1)
df.drop('Thyroid Function', axis=1, inplace=True)

In [860]:
df.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Physical Examination,Adenopathy,Pathology,Focality,Risk,...,T2,T3a,T3b,T4a,T4b,Clinical Hyperthyroidism,Clinical Hypothyroidism,Euthyroid,Subclinical Hyperthyroidism,Subclinical Hypothyroidism
0,27,0,0,0,0,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,34,0,0,1,0,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,30,0,0,0,0,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,62,0,0,0,0,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,62,0,0,0,0,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [861]:
df['Response'].unique()

array(['Indeterminate', 'Excellent', 'Structural Incomplete',
       'Biochemical Incomplete'], dtype=object)

In [862]:
enc = OneHotEncoder(categories='auto')
responce = df['Response'].values.reshape(-1, 1)
enc.fit(responce)
new_features = enc.get_feature_names_out()
print(new_features)
new_responce = pd.DataFrame(enc.transform(responce).toarray())

['x0_Biochemical Incomplete' 'x0_Excellent' 'x0_Indeterminate'
 'x0_Structural Incomplete']


In [863]:
new_responce.columns = ['Biochemical Incomplete', 'Excellent', 'Indeterminate', 'Structural Incomplete']

In [864]:
df = pd.concat([df, new_responce], axis=1)
df.drop('Response', axis=1, inplace=True)

In [865]:
df.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Physical Examination,Adenopathy,Pathology,Focality,Risk,...,T4b,Clinical Hyperthyroidism,Clinical Hypothyroidism,Euthyroid,Subclinical Hyperthyroidism,Subclinical Hypothyroidism,Biochemical Incomplete,Excellent,Indeterminate,Structural Incomplete
0,27,0,0,0,0,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,34,0,0,1,0,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,30,0,0,0,0,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,62,0,0,0,0,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,62,0,0,0,0,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [866]:
df['Stage'].unique()

array(['I', 'II', 'IVB', 'III', 'IVA'], dtype=object)

In [867]:
df['Stage'] = df['Stage'].map({'I': 0, 'II': 1, 'III': 2, 'IVA': 3, 'IVB': 4})

In [868]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          383 non-null    int64  
 1   Gender                       383 non-null    int64  
 2   Smoking                      383 non-null    int64  
 3   Hx Smoking                   383 non-null    int64  
 4   Hx Radiothreapy              383 non-null    int64  
 5   Physical Examination         383 non-null    object 
 6   Adenopathy                   383 non-null    object 
 7   Pathology                    383 non-null    object 
 8   Focality                     383 non-null    object 
 9   Risk                         383 non-null    object 
 10  N                            383 non-null    object 
 11  M                            383 non-null    object 
 12  Stage                        383 non-null    int64  
 13  Recurred            

In [869]:
df['M'] = df['M'].map({'M0': 0, 'M1': 1})

In [870]:
df['N'].unique()

array(['N0', 'N1b', 'N1a'], dtype=object)

In [871]:
enc = OneHotEncoder(categories='auto')
n = df['N'].values.reshape(-1, 1)
enc.fit(n)
new_features = enc.get_feature_names_out()
print(new_features)
new_n = pd.DataFrame(enc.transform(n).toarray())

['x0_N0' 'x0_N1a' 'x0_N1b']


In [872]:
new_n.columns = ['N0', 'N1a', 'N1b']

In [873]:
df = pd.concat([df, new_n], axis=1)
df.drop('N', axis=1, inplace=True)

In [874]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          383 non-null    int64  
 1   Gender                       383 non-null    int64  
 2   Smoking                      383 non-null    int64  
 3   Hx Smoking                   383 non-null    int64  
 4   Hx Radiothreapy              383 non-null    int64  
 5   Physical Examination         383 non-null    object 
 6   Adenopathy                   383 non-null    object 
 7   Pathology                    383 non-null    object 
 8   Focality                     383 non-null    object 
 9   Risk                         383 non-null    object 
 10  M                            383 non-null    int64  
 11  Stage                        383 non-null    int64  
 12  Recurred                     383 non-null    int64  
 13  T1a                 

In [875]:
df['Risk'] = df['Risk'].map({'Low': 0, 'Intermediate': 1, 'High': 2})
df['Focality'] = df['Focality'].map({'Uni-Focal': 0, 'Multi-Focal': 1})

In [876]:
df['Pathology'].unique()

array(['Micropapillary', 'Papillary', 'Follicular', 'Hurthel cell'],
      dtype=object)

In [877]:
enc = OneHotEncoder(categories='auto')
pathology = df['Pathology'].values.reshape(-1, 1)
enc.fit(pathology)
new_features = enc.get_feature_names_out()
print(new_features)
new_pathology = pd.DataFrame(enc.transform(pathology).toarray())

['x0_Follicular' 'x0_Hurthel cell' 'x0_Micropapillary' 'x0_Papillary']


In [878]:
new_pathology.columns = ['Follicular', 'Hurthel cell', 'Micropapillary', 'Papillary']

In [879]:
df = pd.concat([df, new_pathology], axis=1)
df.drop('Pathology', axis=1, inplace=True)

In [880]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          383 non-null    int64  
 1   Gender                       383 non-null    int64  
 2   Smoking                      383 non-null    int64  
 3   Hx Smoking                   383 non-null    int64  
 4   Hx Radiothreapy              383 non-null    int64  
 5   Physical Examination         383 non-null    object 
 6   Adenopathy                   383 non-null    object 
 7   Focality                     383 non-null    int64  
 8   Risk                         383 non-null    int64  
 9   M                            383 non-null    int64  
 10  Stage                        383 non-null    int64  
 11  Recurred                     383 non-null    int64  
 12  T1a                          383 non-null    float64
 13  T1b                 

In [881]:
df['Adenopathy'].unique()

array(['No', 'Right', 'Extensive', 'Left', 'Bilateral', 'Posterior'],
      dtype=object)

In [882]:
enc = OneHotEncoder(categories='auto')
adenopathy = df['Adenopathy'].values.reshape(-1, 1)
enc.fit(adenopathy)
new_features = enc.get_feature_names_out()
print(new_features)
new_Adenopathy = pd.DataFrame(enc.transform(adenopathy).toarray())

['x0_Bilateral' 'x0_Extensive' 'x0_Left' 'x0_No' 'x0_Posterior' 'x0_Right']


In [883]:
new_Adenopathy.columns = ['Bilateral', 'Extensive', 'Left', 'No', 'Posterior', 'Right']

In [884]:
df = pd.concat([df, new_Adenopathy], axis=1)
df.drop('Adenopathy', axis=1, inplace=True)

In [885]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          383 non-null    int64  
 1   Gender                       383 non-null    int64  
 2   Smoking                      383 non-null    int64  
 3   Hx Smoking                   383 non-null    int64  
 4   Hx Radiothreapy              383 non-null    int64  
 5   Physical Examination         383 non-null    object 
 6   Focality                     383 non-null    int64  
 7   Risk                         383 non-null    int64  
 8   M                            383 non-null    int64  
 9   Stage                        383 non-null    int64  
 10  Recurred                     383 non-null    int64  
 11  T1a                          383 non-null    float64
 12  T1b                          383 non-null    float64
 13  T2                  

In [886]:
df['Physical Examination'].unique()

array(['Single nodular goiter-left', 'Multinodular goiter',
       'Single nodular goiter-right', 'Normal', 'Diffuse goiter'],
      dtype=object)

In [887]:
enc = OneHotEncoder(categories='auto')
pe = df['Physical Examination'].values.reshape(-1, 1)
enc.fit(pe)
new_features = enc.get_feature_names_out()
print(new_features)
new_pe = pd.DataFrame(enc.transform(pe).toarray())

['x0_Diffuse goiter' 'x0_Multinodular goiter' 'x0_Normal'
 'x0_Single nodular goiter-left' 'x0_Single nodular goiter-right']


In [888]:
new_pe.columns = ['Diffuse goiter', 'Multinodular goiter', 'Normal', 'Single nodular goiter-left', 'Single nodular goiter-right']

In [889]:
df = pd.concat([df, new_pe], axis=1)
df.drop('Physical Examination', axis=1, inplace=True)

In [890]:
y = df['Recurred'].values
X = df.drop('Recurred', axis=1)

In [891]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(383, 43)
(383, 22)


In [892]:
# build mask
gender_0_mask = df['Gender'] == 0
gender_1_mask = df['Gender'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  312
Male:  71


In [893]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=seed)

In [894]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 4 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 5 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
   Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0     1  Female  0.750000  0.960784  0.039216  0.250000       9      49   
1     2  Female  0.900000  1.000000  0.000000  0.100000       9      53   
2     3  Female  1.000000  1.000000  0.000000  0.000000      10      52   
3     4  Female  0.722222  1.000000  0.000000  0.277778      13      44   
4     5  Female  0.937500  1.000000  0.000000  0.062500      15      46   
5     1    Male  1.000000  0.888889  0.111111  0.000000       6       8   
6     2    Male  0.800000  1.000000  0.000000  0.200000       8       4   
7     3    Male  1.000000  0.875000  0.125000  0.000000       6       7   
8     4    Male  

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [895]:
results_df = pd.concat(results_list, ignore_index=True)

In [896]:
result_path = './results/k13_result.xlsx'
results_df.to_excel(result_path, index=False)

In [897]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.750000,0.960784,0.039216,0.250000,9,49,2,3,...,2,3,0.916667,0.941176,0.058824,0.083333,11,48,3,1
1,2,Female,0.900000,1.000000,0.000000,0.100000,9,53,0,1,...,1,2,0.900000,0.849057,0.150943,0.100000,9,45,8,1
2,3,Female,1.000000,1.000000,0.000000,0.000000,10,52,0,0,...,1,0,1.000000,0.884615,0.115385,0.000000,10,46,6,0
3,4,Female,0.722222,1.000000,0.000000,0.277778,13,44,0,5,...,0,4,1.000000,0.909091,0.090909,0.000000,18,40,4,0
4,5,Female,0.937500,1.000000,0.000000,0.062500,15,46,0,1,...,0,1,0.937500,0.630435,0.369565,0.062500,15,29,17,1


In [898]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(6.5), pvalue=np.float64(0.2358613103342675))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(9.0), pvalue=np.float64(0.440686016488678))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(16.5), pvalue=np.float64(0.44470740102325157))
DT -TPR: MannwhitneyuResult(statistic=np.float64(6.5), pvalue=np.float64(0.2358613103342675))
DT - FPR: TtestResult(statistic=np.float64(-1.2108239968911128), pvalue=np.float64(0.28467653380494085), df=np.float64(4.583154987971199))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(15.5), pvalue=np.float64(0.5888091865408535))
RF -TPR: MannwhitneyuResult(statistic=np.float64(7.0), pvalue=np.float64(0.27964153403730163))
RF - FPR: MannwhitneyuResult(statistic=np.float64(7.0), pvalue=np.float64(0.26520539259150755))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(16.0), pvalue=np.float64(0.5165484841465027))
LR -TPR: MannwhitneyuResult(statistic=np.float64(7.0), pvalue=np.float64(0.279641534037301